In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import Callback
import tensorflow_datasets as tfds
(dataset, dataset_info) = tfds.load('cats_vs_dogs', split='train[:90%]', with_info=True, as_supervised=True)
num_classes = dataset_info.features['label'].num_classes
def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224))
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.2, upper=1.8)
    image = tf.image.random_hue(image, max_delta=0.1)
    image = tf.image.random_saturation(image, lower=0.2, upper=1.8)
    image = tf.image.per_image_standardization(image)
    label = tf.one_hot(label, num_classes)
    return image, label
dataset = dataset.map(preprocess_image)
num_samples = dataset_info.splits['train[:90%]'].num_examples
num_train_samples = int(0.8 * num_samples)
train_dataset = dataset.take(num_train_samples)
val_dataset = dataset.skip(num_train_samples)
batch_size = 32
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)
class TrainingAccuracyCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch + 1}/{self.params['epochs']} - Training Accuracy: {logs['accuracy']:.4f}")
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    #Dropout(0.5),- Increased accuracy after commenting this
    Dense(num_classes, activation='softmax')
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
accuracy_callback = TrainingAccuracyCallback()
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset, callbacks=[accuracy_callback])
final_training_accuracy = history.history['accuracy'][-1]
print("Training accuracy:", final_training_accuracy)
#Here is my attempt to see how much training accuracy we are getting after training the model to ensure that it is ready for training prior to unseen data

Epoch 1/10
524/524 [==============================] - 132s 247ms/step - loss: 0.7032 - accuracy: 0.5911 - val_loss: 0.5684 - val_accuracy: 0.6920
Epoch 2/10
524/524 [==============================] - 126s 241ms/step - loss: 0.5162 - accuracy: 0.7415 - val_loss: 0.4831 - val_accuracy: 0.7603
Epoch 3/10
524/524 [==============================] - 147s 280ms/step - loss: 0.4276 - accuracy: 0.8022 - val_loss: 0.4371 - val_accuracy: 0.7904
Epoch 4/10
524/524 [==============================] - 125s 239ms/step - loss: 0.3508 - accuracy: 0.8485 - val_loss: 0.4563 - val_accuracy: 0.7887
Epoch 5/10
524/524 [==============================] - 127s 242ms/step - loss: 0.2805 - accuracy: 0.8873 - val_loss: 0.4830 - val_accuracy: 0.7985
Epoch 6/10
524/524 [==============================] - 125s 238ms/step - loss: 0.2146 - accuracy: 0.9178 - val_loss: 0.5723 - val_accuracy: 0.8023
Epoch 7/10
524/524 [==============================] - 127s 242ms/step - loss: 0.1604 - accuracy: 0.9418 - val_loss: 0.6213 -